In [ ]:
# Mount my drive in google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Extract zip file of all Pokemon images
from zipfile import ZipFile
file_name = "./drive/MyDrive/pokemon/augmented.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
pokemon_classes =[
    "Abra", "Aerodactyl", "Alakazam", "Arbok", "Arcanine", "Articuno",
    "Beedrill", "Bellsprout", "Blastoise", "Bulbasaur", "Butterfree",
    "Caterpie", "Chansey", "Charizard", "Charmander", "Charmeleon", "Clefable",
    "Clefairy", "Cloyster", "Cubone",
    "Dewgong", "Diglett", "Ditto", "Dodrio", "Doduo", "Dragonair", "Dragonite",
    "Dratini", "Drowzee", "Dugtrio",
    "Eevee", "Ekans", "Electabuzz", "Electrode", "Exeggcute", "Exeggutor",
    "Farfetchd", "Fearow", "Flareon", 
    "Gastly", "Gengar", "Geodude", "Gloom", "Golbat", "Goldeen", "Golduck",
    "Golem", "Graveler", "Grimer", "Growlithe", "Gyarados",
    "Haunter", "Hitmonchan", "Hitmonlee", "Horsea", "Hypno", 
    "Ivysaur",
    "Jigglypuff", "Jolteon", "Jynx",
    "Kabuto", "Kabutops", "Kadabra", "Kakuna", "Kangaskhan", "Kingler", "Koffing","Krabby",
    "Lapras", "Lickitung", 
    "Machamp", "Machoke", "Machop", "Magikarp", "Magmar", "Magnemite", "Magneton", "Mankey",
    "Marowak","Meowth", "Metapod", "Mew", "Mewtwo", "Moltres", "MrMime", "Muk",
    "Nidoking", "Nidoqueen", "Nidoran(Female)", "Nidoran(Male)", "Nidorina", "Nidorino", "Ninetales",
    "Oddish", "Omanyte", "Omastar", "Onix", 
    "Paras", "Parasect", "Persian", "Pidgeot", "Pidgeotto", "Pidgey", "Pikachu", "Pinsir", "Poliwag",
    "Poliwhirl","Poliwrath", "Ponyta", "Porygon", "Primeape", "Psyduck",
    "Raichu", "Rapidash", "Raticate", "Rattata", "Rhydon", "Rhyhorn",
    "Sandshrew", "Sandslash", "Scyther", "Seadra", "Seaking", "Seel", "Shellder", "Slowbro", "Slowpoke",
    "Snorlax", "Spearow", "Squirtle", "Starmie", "Staryu",
    "Tangela", "Tauros", "Tentacool", "Tentacruel",
    "Vaporeon", "Venomoth", "Venonat", "Venusaur", "Victreebel", "Vileplume", "Voltorb", "Vulpix",
    "Wartortle", "Weedle", "Weepinbell", "Weezing", "Wigglytuff",
    "Zapdos", "Zubat"    
]


In [ ]:
# Show barchart of number of images in each training class
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib

nimgs = {}
for i in range(0,151):
    nimages = len(os.listdir('/content/prac_colab/train/'+pokemon_classes[i]+'/'))
    nimgs[pokemon_classes[i]]=nimages
plt.figure(figsize=(151, 6))
plt.bar(range(len(nimgs)), list(nimgs.values()), align='center')
plt.xticks(range(len(nimgs)), list(nimgs.keys()))
plt.title('Distribution of different classes in Training Dataset')
plt.show()

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_width= 256
img_height= 256
batch_size=32

In [ ]:
# Normalize the images in the training folder
TRAINING_DIR = '/content/augmented/train/'
train_data_gen = ImageDataGenerator(rescale = 1/255.0)

train_gen = train_data_gen.flow_from_directory(TRAINING_DIR, batch_size=batch_size, class_mode='categorical',target_size=(img_height, img_width))

In [ ]:
# Normalize the images in the validation folder
VALIDATION_DIR = '/content/augmented/validate/'
validation_data_gen = ImageDataGenerator(rescale = 1/255.0)

validation_gen = validation_data_gen.flow_from_directory(VALIDATION_DIR, batch_size=batch_size, class_mode='categorical', target_size=(img_height, img_width))

In [ ]:
# autosave best Model
callbacks = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
best_model_file = '/content/CNN_aug_best_weights.h5'
best_model = keras.callbacks.ModelCheckpoint(best_model_file, monitor='val_acc', verbose = 1, save_best_only = True)

In [ ]:
kern_size = 3
epochs = 10

In [ ]:
# Create self-constructed CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size= kern_size, activation='relu', input_shape=(img_height, img_width, 3)),
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Conv2D(32, kernel_size= kern_size, activation='relu', input_shape=(img_height, img_width, 3)), 
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Conv2D(32, kernel_size= kern_size, activation='relu', input_shape=(img_height, img_width, 3)), 
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Conv2D(32, kernel_size= kern_size, activation='relu', input_shape=(img_height, img_width, 3)), 
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(15, activation='softmax')
])

In [ ]:
from tensorflow.keras.applications import EfficientNetB7, DenseNet201, VGG19, InceptionV3

In [ ]:
# Setup pre-trained model
pretrained_layers = VGG19(include_top = False, weights = 'imagenet', input_shape = (img_height,img_width,3))

for layer in pretrained_layers.layers[:19]:
    layer.trainable = False

In [ ]:
pretrained_model = keras.Sequential()
pretrained_model.add(keras.layers.InputLayer(input_shape=(img_height, img_width, 3)))
pretrained_model.add(pretrained_layers)
pretrained_model.add(keras.layers.Flatten())
pretrained_model.add(keras.layers.Dense(151, activation='softmax'))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
pretrained_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
# Train the pretrained model
history_pretrained = pretrained_model.fit(train_gen, epochs=epochs, verbose=1, validation_data=validation_gen, callbacks = [best_model])

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
# Train the self-constructed model
history = model.fit(train_gen, epochs=epochs, verbose=1, validation_data=validation_gen, callbacks = [best_model])

In [ ]:
# Normalize the images in the test set
TEST_DIR = '/content/augmented/test/'
test_data_gen = ImageDataGenerator(rescale = 1/255.0)

test_gen = test_data_gen.flow_from_directory(TEST_DIR, batch_size=batch_size, class_mode='categorical',target_size=(img_height, img_width), shuffle=False)


In [ ]:
# Test self-contructed model on the test set
model.evaluate(x=test_gen)

In [ ]:
# Test pretrained model on the test set
pretrained_model.evaluate(x=test_gen)

In [ ]:
# Test models with new random Pokemon image
import numpy as np
import cv2

poke_pic = '/content/charmander.jpg'
new_img = cv2.imread(poke_pic)/255.0
new_img = cv2.resize(new_img, (img_width, img_height))
new_img = np.reshape(new_img, [1, img_width, img_height, 3])

In [ ]:
model_predictions = model.predict(new_img)
print(model_predictions)
print(pokemon_classes[np.argmax(model_predictions)])

In [ ]:
pre_predictions = pretrained_model.predict(new_img)
print(pre_predictions)
print(pokemon_classes[np.argmax(pre_predictions)])

In [ ]:
# Save the pretrained model
pretrained_model.save('/content/151_model_pre_vgg19_10epoch')

In [ ]:
# Zip the model files
import shutil
model_file = '/content/151_model_pre_vgg19_10epoch'

shutil.make_archive('151_model_10epoch_vgg19_files', 'zip', model_file)

In [ ]:
# Import the model
from zipfile import ZipFile
model_file_name = "./drive/MyDrive/pokemon/151_model_10epoch_vgg19_files.zip"

with ZipFile(model_file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
# Load the model
model_151 = tf.keras.models.load_model('./model_151')

In [ ]:
# Make predictions on the test set
vgg19_predictions = model_151.predict_generator(test_gen,650)

In [ ]:
# Create confusion matrix and print out the classification report for all of the classes based on the test set
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_predictions = np.argmax(vgg19_predictions, axis=1)
matrix = confusion_matrix(test_gen.classes, y_predictions)
class_report = classification_report(test_gen.classes, y_predictions,target_names=pokemon_classes)
print(class_report)

                 precision    recall  f1-score   support

           Abra       0.99      1.00      1.00       118
     Aerodactyl       0.96      0.99      0.98       119
       Alakazam       0.93      0.96      0.94       123
          Arbok       1.00      0.99      1.00       123
       Arcanine       1.00      0.85      0.92       122
       Articuno       1.00      0.99      1.00       135
       Beedrill       0.98      1.00      0.99       120
     Bellsprout       1.00      0.80      0.89       120
      Blastoise       0.98      1.00      0.99       126
      Bulbasaur       1.00      1.00      1.00       123
     Butterfree       1.00      0.99      0.99        96
       Caterpie       0.99      1.00      1.00       119
        Chansey       0.98      1.00      0.99       129
      Charizard       1.00      0.97      0.98       119
     Charmander       1.00      1.00      1.00       151
     Charmeleon       0.99      1.00      0.99       137
       Clefable       1.00    

In [ ]:
# Print out number of false positives, false negatives, true postives and true negatives
FP = matrix.sum(axis=0) - np.diag(matrix)  
FN = matrix.sum(axis=1) - np.diag(matrix)
TP = np.diag(matrix)
TN = matrix.sum() - (FP + FN + TP)

print("False Positives: "+str(FP))
print("False Negatives: "+str(FN))
print("True Positives: "+str(TP))
print("True Negatives: "+str(TN))

In [ ]:
# Most common Pokemon class labeling mistake (True class, Incorrect Label used, number of occurrences)
max_value = -1
x_coord = -1
y_coord = -1
for i in range(0, matrix.shape[0]):
  for j in range(0, matrix.shape[1]):
    if i != j:
      if matrix[i][j] > max_value:
        max_value = matrix[i][j]
        x_coord = i
        y_coord = j

print(pokemon_classes[x_coord])
print(pokemon_classes[y_coord])
print(max_value)

Pidgeot
Pidgeotto
12


In [ ]:
# Print out the Pokemon classs with the highest number of incorrect predictions made for it (Class, number of incorrect predictions)
x_row = -1
max_x_row = -1
for i in range(0, matrix.shape[0]):
  row_sum = matrix[i].sum()
  wrong_sum = row_sum - matrix[i][i]

  if wrong_sum > max_x_row:
    x_row = i
    max_x_row = wrong_sum

print(pokemon_classes[x_row])
print(max_x_row) 

Exeggutor
42


In [ ]:
# Print out the Pokemon class label used for the most number of incorrect predictions
y_col = -1
max_y_col = -1
summed_columns = np.sum(matrix, axis=0)
for i in range(0, matrix.shape[1]):
  row_sum = summed_columns[i]
  wrong_sum = row_sum - matrix[i][i]

  if wrong_sum > max_y_col:
    y_col = i
    max_y_col = wrong_sum

print(summed_columns)
print(pokemon_classes[y_col])
print(max_y_col)
print(summed_columns[y_col])


[119 123 127 122 104 134 122  96 128 123  95 120 131 115 151 139 119 134
 143 116 133 116 120 119 124 128 134 128 118 152 120 120 125 151 130 107
 142 144 140 132 146 118 144 141 138 129 138 120 128 136 143 144 139 146
 154 139 121 138 152 149 144 137 137 130 151 142 145 140 157 141 158 126
 124 132 130 135 122 164 106 119 158 155 155 127 135 131 154 154 115 118
 126 153 139 153 135 126 122 141 127 120 111 143 145 186 122 153 159 108
 133 138 143 121 154 147 128 130 146 121 136 136 171 148 137 141 136 137
 122 199 201 154 144 141 155 163 135 141 174 156 141 200 144 166 164 135
 141 146 134 149 171 151 126]
Spearow
38
201
